In [ ]:
import pandas as pd
import numpy as np
import seaborn as sbs
import matplotlib.pyplot as plt

In [ ]:
input_csv = "../results/math.csv"
baseline = "x86_64,qemu,qemu"
base_arch, base_runtime, base_tag = baseline.split(',')
df = pd.read_csv(input_csv, sep=';')
df

In [ ]:
base_df = df.loc[df['tag'] == 'qemu']
base_df

In [ ]:
# Get the mean for each baseline benchmark
base_means = {}
for b in set(base_df['bench']):
    base_means[b] = np.array(base_df.loc[base_df['bench'] == b]['value'].values, dtype=np.float32).mean()
base_means

In [ ]:
# Print the mean of every benchmark for each runtime
mean_df = pd.DataFrame()
for b in sorted(set(df['bench'])):
    df_b = df.loc[df['bench'] == b]
    tmp_dict = { 'bench': b }
    for t in set(df_b['tag']):
        df_b_t = df_b.loc[df_b['tag'] == t]
        tmp_dict[t] = np.mean(df_b_t['value'])
    mean_df = mean_df.append(tmp_dict, ignore_index=True)
mean_df

In [ ]:
# Normalize all results from original df to these means
df_norm = pd.DataFrame(columns=['arch', 'bench', 'dataset', 'threads', 'unit', 'value', 'runtime',
                                'tag', 'norm', 'label'])
norm_vals = []
for row in df.itertuples():
    try:
        if row.arch == base_arch and row.runtime == base_runtime and row.tag == base_tag:
            continue
        # norm = base_means[row.bench] / float(row.value)      # speedup
        norm = float(row.value) / base_means[row.bench]    # relative perf
        
        # norm = 100 * (base_means[row.bench] - float(row.value)) / base_means[row.bench]
        dct = row._asdict()
        dct['norm'] = norm
        dct['label'] = f"{dct['tag']}"
        dct['bench'] = dct['bench'][11:]
        # dct['label'] = f"{dct['runtime']}-{dct['tag']}"
        del dct['Index']
        del dct['cmdline']
        norm_vals.append(dct)
    except KeyError:
        pass
df_norm = df_norm.append(norm_vals, ignore_index=True)
df_norm

In [ ]:
import re 

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [ ]:
max(df_norm['norm'].values)
df_norm.loc[df_norm['bench'] == 'log']['norm'].values.mean()

In [ ]:
fig = plt.figure(figsize=(5,2.5), dpi=500)
sbs.set(style="whitegrid")
palette = {
    'orange': '#faa200',
    'sky blue': '#00b7ec',
    'bluish green': '#00a077',
    'yellow': '#f5e636',
    'blue': '#0077b8',
    'vermillion': '#f3640d',
    'reddish purple': '#e47ead'
}
order = [ 'sqrt', 'exp', 'log', 'cos', 'sin', 'tan', 'acos', 'asin', 'atan' ]
ax = sbs.barplot(data=df_norm, x='bench', y='norm', hue='label',
                 hue_order=['risotto', 'native'], order=order)
plt.grid(visible=True, axis='y')
plt.xticks(rotation=0, ha="center", fontsize='xx-small')
plt.xlabel("Function", fontsize='x-small')
plt.ylabel("Speedup w.r.t. QEMU")
max_val = max(df_norm['norm'].values)
plt.ylim(0, max_val*1.1)
plt.axhline(y=1, xmin=0, xmax=1, color='tomato', linewidth=2.5)
# Annotate the raw value of the baseline
for idx, value in enumerate(order):
    bm = base_means[f"micro.math-{value}"]
    if bm > 1000000:
        v = f"{bm / 1000000:.0f}M"
    elif bm > 1000:
        v = f"{bm / 1000:.1f}k"
    else:
        v = f"{bm:.0f}"
    #tmp = max(mean_df.loc[mean_df['bench'] == value].values[0][1:]) / base_means[value]
    tmp = max(df_norm.loc[df_norm['bench'] == value]['norm'].values)
    plt.text(idx, tmp+.5, f"{v}", fontsize='xx-small', color='tomato', ha='center')
#plt.vlines(range(0, len(order)), ymin=0, ymax=max_val, linestyle='dashed', colors='grey', linewidth=.25, zorder=0)

# Set color + hatch
style = {
    'fill': [ True, True ],
    'color': [ palette['bluish green'], palette['orange'] ],
    'hatch': [ '', '', ''],
    'edgecolor': [ 'black', 'black', 'black' ]
}
for idx, bar in enumerate(ax.patches):
    bar_nr = int(idx / int(len(order)))
    bar.set(color=style['color'][bar_nr], fill=style['fill'][bar_nr],
            hatch=style['hatch'][bar_nr], edgecolor=style['edgecolor'][bar_nr])

plt.legend(loc='upper center', bbox_to_anchor=(.5, 1.15), borderaxespad=0, ncol=2, fontsize='x-small')

In [ ]:
fig.savefig(f"{input_csv[:-3]}pdf", dpi=500, bbox_inches='tight')

In [ ]:
for i in sorted_nicely(base_means):
    print(f"{i:25}: {base_means[i]:>20.1f}")

In [ ]:
print(f"bench{20*' '}    risotto     native")
for b in sorted_nicely(base_means):
    ris = df_norm.loc[(df_norm['bench'] == b[11:]) & (df_norm['tag'] == 'risotto')]
    nat = df_norm.loc[(df_norm['bench'] == b[11:]) & (df_norm['tag'] == 'native')]
    print(f"{b:25} {np.mean(ris['norm'].values):>10.4f} {np.mean(nat['norm'].values):>10.2f}")